# Predicting Flagellate concentration with functional clustering and regression (daily means - removed seasonality before training)

## Importing

In [51]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xskillscore as xs

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import r_regression

from skfda.representation.grid import FDataGrid
from skfda.ml.clustering import KMeans

from skfda.ml.regression import FPLSRegression

from sklearn.metrics import root_mean_squared_error as rmse

import os
import lzma
import dill

import random

import cmocean.cm as cm
import salishsea_tools.viz_tools as sa_vi

np.warnings.filterwarnings('ignore') # For the nan mean warning


## Datasets Preparation

In [52]:
# Creation of the training - testing datasets
def datasets_preparation(dataset, dataset2, clusters, name):
    
    indx = np.where((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
    
    targets = dataset[name].to_numpy().reshape(*dataset[name].to_numpy().shape[:1],-1)

    inputs = np.stack([
        dataset2['Summation_of_solar_radiation'].to_numpy().reshape(*dataset2['Summation_of_solar_radiation'].to_numpy().shape[:1],-1),
        dataset2['Mean_wind_speed'].to_numpy().reshape(*dataset2['Mean_wind_speed'].to_numpy().shape[:1],-1),
        dataset2['Mean_air_temperature'].to_numpy().reshape(*dataset2['Mean_air_temperature'].to_numpy().shape[:1],-1)
        ])

    # Deleting 29 of February
    inputs = np.delete(inputs,indx,axis=1)
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    inputs = np.split(inputs,len(np.unique(dataset.time_counter.dt.year)),axis=1)
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    inputs = np.nanmean(inputs,axis=0)
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset2.x, len(dataset2.y))
    y =  np.tile(np.repeat(dataset2.y, len(dataset2.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0)) & (x>10) & ((x>100) | (y<880)))
    inputs = inputs[:,:,indx[0]]
    targets = targets[:,indx[0]]

    clusters = np.tile(np.ravel(clusters), len(dataset.time_counter))
    clusters = clusters[indx[0]]

    return(inputs, targets, indx, clusters)


## Clustering Finalization

In [53]:
def clustering(dataset,quant,indx,name):

    # Training
    n_clusters = 6
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(quant)

    # Sorting so that cluster 1 has the minimum mean target value, 6 the maximum

        # Finding the mean of each cluster
    if name == 'inputs':
        cluster_mean_all = np.mean(kmeans.cluster_centers_.data_matrix,axis=1)
        cluster_mean = cluster_mean_all[:,0]  # Sorted based on the first input
    else:
        cluster_mean = np.squeeze(np.mean(kmeans.cluster_centers_.data_matrix,axis=1))

        # The index to sort the clusters
    indx3 = np.argsort(np.argsort(cluster_mean)) # For the complete map we need the double np.argsort

        # Sorting
    for j in np.arange(0,len(np.unique(clusters))):
        clusters = xr.where(kmeans.labels_==j, indx3[j], clusters)

    unique, _ = np.unique(clusters, return_counts=True)

    # Creating the map
    indx2 = np.full(len(dataset.y) * len(dataset.x),np.nan)
    indx2[indx[0]] = clusters
    clusters = np.reshape(indx2,(len(dataset.y),len(dataset.x))) 
    clusters2 = xr.DataArray(clusters,dims = ['y','x'])
    
    # Plotting
    fig, ax = plt.subplots(figsize =(5,9))

    cmap = plt.get_cmap('tab20', unique.max()+1)
    cmap.set_bad('gray')
    clus = clusters2.plot(ax=ax, cmap=cmap, vmin = unique.min(), vmax = unique.max()+1, add_colorbar=False)

    cbar = fig.colorbar(clus, ticks = unique+0.5) 
    cbar.set_ticklabels(unique+1)
    cbar.set_label('Clusters [count]')
    ax.set_title('Functional Clustering for '+ name + ' (2007-2020)')

    sa_vi.set_aspect(ax)
    plt.show()

    return(clusters,n_clusters)
    

## Functional Clustering (Target)

In [54]:
def func_clust_target(dataset, name):

    indx = np.where((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
    
    targets = dataset[name].to_numpy().reshape(*dataset[name].to_numpy().shape[:1],-1)

    # Deleting 29 of February
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset.x, len(dataset.y))
    y =  np.tile(np.repeat(dataset.y, len(dataset.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0))& (x>10) & ((x>100) | (y<880)))
    targets = targets[:,indx[0]]

    # Converting it to an appropriate format for functional clustering
    targets = targets.transpose()
    targets2 = FDataGrid(targets)

    clusters, n_clusters = clustering(dataset,targets2,indx,name)

    return(clusters,0,n_clusters)


## Functional Clustering (Drivers)

In [55]:
def func_clust_drivers(dataset,dataset2,name):

    indx = np.where((dataset2.time_counter.dt.month==2) & (dataset2.time_counter.dt.day==29))

    inputs = np.stack([
        dataset2['Summation_of_solar_radiation'].to_numpy().reshape(*dataset2['Summation_of_solar_radiation'].to_numpy().shape[:1],-1),
        dataset2['Mean_wind_speed'].to_numpy().reshape(*dataset2['Mean_wind_speed'].to_numpy().shape[:1],-1),
        dataset2['Mean_air_temperature'].to_numpy().reshape(*dataset2['Mean_air_temperature'].to_numpy().shape[:1],-1)
        ])
    
    targets = dataset['Diatom'].to_numpy().reshape(*dataset2['Summation_of_solar_radiation'].to_numpy().shape[:1],-1)

    # Deleting 29 of February
    inputs = np.delete(inputs,indx,axis=1)
    targets = np.delete(targets,indx,axis=0)

    # Splitting in years
    inputs = np.split(inputs,len(np.unique(dataset2.time_counter.dt.year)),axis=1)
    targets = np.split(targets,len(np.unique(dataset.time_counter.dt.year)),axis=0)

    # Means
    inputs = np.nanmean(inputs,axis=0)
    targets = np.nanmean(targets,axis=0)

    x =  np.tile(dataset2.x, len(dataset2.y))
    y =  np.tile(np.repeat(dataset2.y, len(dataset2.x)),1)

    indx = np.where((~np.isnan(targets).any(axis=0))& (x>10) & ((x>100) | (y<880))) # Target goes down to 100m
    inputs = inputs[:,:,indx[0]]

    # Scaling the inputs
    temp = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp = temp.transpose()
    scaler_inputs = make_column_transformer((MinMaxScaler(), [0,1,2]))
    temp = scaler_inputs.fit_transform(temp)
    temp = temp.transpose()
    inputs = np.reshape(temp,(len(inputs),inputs.shape[1],inputs.shape[2])) 

    # Converting it to an appropriate format for functional clustering
    inputs = np.transpose(inputs,axes=(2,1,0))
    inputs2 = FDataGrid(inputs, np.arange(0,len(inputs[0])))

    clusters, n_clusters = clustering(dataset2,inputs2,indx,name)

    return(clusters, 1, n_clusters)


## File Creation

In [56]:
def file_creation(path, variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path = path + 'targets_predictions.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})
    

## Regressor

In [57]:
def regressor (inputs, targets, name, cluster):

# Printing of the correlation coefficients
    temp_inputs = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp_inputs = temp_inputs.transpose()
    temp_targets = np.ravel(targets)

    r = np.round(r_regression(temp_inputs,temp_targets),2)
    dict = {'Summation_of_solar_radiation':r[0], 'Mean_wind_speed':r[1], 'Mean_air_temperature':r[2]}
    
    print('The correlation coefficients between each input and ' + name +  ' for cluster ' + str(cluster+1) + ' are: ' +str(dict))

    # Scaling the inputs
    scaler_inputs = make_column_transformer((MinMaxScaler(), [0,1,2]))
    temp_inputs = scaler_inputs.fit_transform(temp_inputs)
    temp_inputs = temp_inputs.transpose()
    inputs = np.reshape(temp_inputs,(len(inputs),inputs.shape[1],inputs.shape[2]))   
    inputs = np.transpose(inputs,axes=(2,1,0))
    
    # Scaling the targets
    scaler_targets = StandardScaler()
    temp_targets = np.expand_dims(temp_targets,-1)
    temp_targets = scaler_targets.fit_transform(temp_targets)
    targets = temp_targets.reshape(targets.shape)

    # Final transformations
    targets = targets.transpose()
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets[0])))
    # targets = FDataGrid(data_matrix=targets, grid_points=np.arange(0,len(targets)))

    ## Smoothing
    # targets = targets.to_basis(FourierBasis(n_basis=10)

    model = FPLSRegression(n_components=14)
    regr = model.fit(inputs,targets)

    return(regr,scaler_inputs,scaler_targets)


## Seasonality

In [58]:
def seasonality(n_clusters,targets,season_id):  

    # Season id 0 is one seasonality, 1 is one for each cluster
    
    if season_id == 0:

        season = np.mean(targets,axis=(1,2))
        plt.plot(season)
        plt.suptitle('Long-term seasonality (2007-2020)')

        season_train = np.tile(season,14) # Broadcasting season to all training years
        season_train = np.reshape(season_train,(75,14),order='F')

        season_test = np.tile(season,4) # Broadcasting season to all testing years
        season_test = np.reshape(season_test,(75,4),order='F')
            
    else:

        season = np.mean(targets,axis=1)

        for i in range (0, n_clusters):

            plt.plot(season[:,i])
            plt.legend(('Cluster 1','Cluster 2','Cluster 3','Cluster 4','Cluster 5','Cluster 6'))
            plt.suptitle('Long-term seasonality (2007-2020)')

        season_train = np.tile(season,14) # Broadcasting season to all training years
        season_train = np.reshape(season_train,targets.shape)

        season_test = np.tile(season,4) # Broadcasting season to all testing years
        season_test = np.reshape(season_test,(75,4,6))
    
    return(season_train,season_test)
            

## Scaling

In [59]:
def scaling(regr,inputs,scaler_inputs,targets,scaler_targets):

    # Scaling the inputs
    temp = np.reshape(inputs,(len(inputs),inputs.shape[1]*inputs.shape[2]))
    temp = temp.transpose()
    temp = scaler_inputs.transform(temp)
    temp = temp.transpose()        
    inputs = np.reshape(temp,(len(inputs),inputs.shape[1],inputs.shape[2]))
        
    inputs = np.transpose(inputs,axes=(2,1,0))
    inputs = FDataGrid(data_matrix=inputs, grid_points=np.arange(0,len(targets)))

    predictions = regr.predict(inputs)

    # # Post-processing of predictions
    # predictions = np.array(predictions.to_grid(np.arange(0,len(targets))).data_matrix)
    # predictions = np.squeeze(predictions,2)

    # Scaling the predictions
    temp = np.ravel(predictions)
    temp = np.expand_dims(temp,axis=-1)
    temp = scaler_targets.inverse_transform(temp)
    predictions = temp.reshape(predictions.shape)
    predictions = predictions.transpose()

    return(predictions)


## Scatter Plot

In [60]:
def scatter_plot(dates,targets, predictions, name):

    indx = pd.DatetimeIndex(dates[0:75]) # From the first year

    # compute slope m and intercept b
    m, b = np.polyfit(targets, predictions, deg=1)

    fig, ax = plt.subplots()

    scatter = ax.scatter(targets,predictions, s = 10, c= indx.month)

    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
        np.max([ax.get_xlim(), ax.get_ylim()])]

    # plot fitted y = m*x + b
    ax.axline(xy1=(0, b), slope=m, color='r')

    ax.set_xlabel('targets')
    ax.set_ylabel('predictions')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_aspect('equal')
    ax.legend(handles=scatter.legend_elements()[0], labels=['February','March','April'])

    ax.plot(lims, lims,linestyle = '--',color = 'k')

    fig.suptitle(name)

    plt.show()

    return(m)

## Pre-training

In [61]:
def pre_training(dataset,dataset2,n_clusters,clusters0,name):

    np.warnings.filterwarnings('ignore') # For the nan mean warning

    clusters_indiv_t = np.zeros((len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),n_clusters))
    clusters_indiv_d = np.zeros((3,len(np.unique(dataset.time_counter.dt.dayofyear))-1,len(np.unique(dataset.time_counter.dt.year)),n_clusters))

    ds = dataset
    ds2 = dataset2

    for i in range(0, len(np.unique(ds.time_counter.dt.year))):

        dataset = ds.sel(time_counter = slice(str(np.unique(ds.time_counter.dt.year)[i]), str(np.unique(ds.time_counter.dt.year)[i])))
        dataset2 = ds2.sel(time_counter = slice(str(np.unique(ds2.time_counter.dt.year)[i]), str(np.unique(ds2.time_counter.dt.year)[i])))

        inputs, targets, indx, _ = datasets_preparation(dataset, dataset2, clusters0, name)

        inputs1 = np.squeeze(inputs)
        targets1 = np.squeeze(targets)
        clusters1 = np.ravel(clusters0)[indx]

        for j in range (0,n_clusters):

            temp = xr.where(clusters1==j, inputs1, np.nan)
            clusters_indiv_d[:,:,i,j] = np.nanmean(temp,axis=2)

            temp = xr.where(clusters1==j, targets1, np.nan)
            clusters_indiv_t[:,i,j] = np.nanmean(temp,axis=1)

    return(clusters_indiv_d,clusters_indiv_t,indx)


## Plotting (Mean Values)

In [62]:
def plotting_mean_values(years,n_clusters,targets,predictions,r_train,rms_train,slope_train,category,units,region):

    ticks = []
    for i in range (0,targets.shape[0]*targets.shape[1],targets.shape[0]):
        ticks.append(i)

    targets = np.reshape(targets,(targets.shape[0]*targets.shape[1],targets.shape[2]), order = 'F')
    predictions = np.reshape(predictions,(predictions.shape[0]*predictions.shape[1],predictions.shape[2]), order = 'F')
    
    targets_masked = np.ma.array(targets)
    predictions_masked = np.ma.array(predictions)

    targets_masked[ticks] = np.ma.masked
    predictions_masked[ticks] = np.ma.masked

    for i in range (0,n_clusters):

        fig, _ = plt.subplots(figsize=(19,5))

        if region == 'Salish Sea (Training)':
            temp = pd.DataFrame(np.vstack((r_train[i],rms_train[i],slope_train[i])).transpose(),index=['Cluster '+str(i+1)],columns=['r_train','rms_train','slope_train'])
            display(temp)

        plt.plot(targets_masked[:,i], label = 'targets')
        plt.plot(predictions_masked[:,i], label = 'predictions')
        plt.xlabel('Years')
        plt.xticks(ticks,years)
        plt.suptitle('Mean '+category + ' ' +units + ' (15 Feb - 30 Apr) ' + region + ' (Cluster '+str(i+1)+ ')')
        plt.legend()

        plt.show()


## Post-Training

In [63]:
def post_training(n_clusters,targets,predictions):

    r_train = np.full(n_clusters,np.nan)
    rms_train = np.full(n_clusters,np.nan)
    slope_train = np.full(n_clusters,np.nan)

    for i in range (0,n_clusters):

        r_train[i] = np.round(np.corrcoef(targets[:,:,i],predictions[:,:,i])[0][1],3)
        rms_train[i] = rmse(targets[:,:,i],predictions[:,:,i])
        m,_ = np.polyfit(np.ravel(targets[:,:,i]),np.ravel(predictions[:,:,i]), deg=1)
        slope_train[i] = np.round(m,3)

    return(r_train,rms_train,slope_train)


## Evaluation

In [64]:
def evaluation (dates,name,years,targets,predictions,id):

    # For every year
    r_years = np.array([])
    rms_years = np.array([])
    slope_years = np.array([])

    ticks = []
    for i in range (0,targets.shape[0]*targets.shape[1],targets.shape[0]):
        ticks.append(i)
        
    for i in range (0, len(years)):

        r_year = np.round(np.corrcoef(np.ravel(targets[:,i]), np.ravel(predictions[:,i]))[0][1],3)
        rms_year = np.round(rmse(np.ravel(targets[:,i]), np.ravel(predictions[:,i])),5)
        m,_ = np.polyfit(np.ravel(targets[:,i]), np.ravel(predictions[:,i]), deg=1)
        slope_year = np.round(m,3)
        
        print ('The correlation coefficient during testing for year '+ str(years[i]) + ' is: ' + str(r_year))
        print ('The rmse during testing for year '+ str(years[i]) + ' is: ' + str(rms_year))
        print ('The slope of the best fitting line during testing for year '+ str(years[i]) + ' is: ' + str(slope_year))

        # _ = scatter_plot(dates, targets[:,i], predictions[:,i], name + ' for '+ str(years[i]) + ' (Cluster ' + str(id) + ')') 

        r_years = np.append(r_years,r_year)
        rms_years = np.append(rms_years,rms_year)
        slope_years = np.append(slope_years,slope_year)

    return(r_years, rms_years, slope_years)


## Plotting (Clusters)

In [65]:
def plotting_clusters(years,n_clusters,targets):

    ticks = []
    for i in range (0,targets.shape[0]*targets.shape[1],targets.shape[0]):
        ticks.append(i)

    targets = np.reshape(targets,(1050,6), order = 'F')
    
    targets_masked = np.ma.array(targets)

    targets_masked[ticks] = np.ma.masked

    for i in range (0,n_clusters):

        fig, _ = plt.subplots(figsize=(19,5))

        plt.plot(targets_masked[:,i])
        plt.xlabel('Years')
        plt.xticks(ticks,years)
        plt.suptitle('Cluster '+str(i+1))

        plt.show()
        

## Training

In [ ]:
name = 'Diatom'
units = '[mmol m-2]'
category = 'Concentrations'

ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

ds = ds.isel(y=(np.arange(ds.y[0], ds.y[-1], 5)), 
    x=(np.arange(ds.x[0], ds.x[-1], 5)))

ds2 = ds2.isel(y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
    x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

dataset = ds.sel(time_counter = slice('2007', '2020'))
dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

years = np.unique(dataset.time_counter.dt.year)

# Selecting the clustering input (drivers or target)
clusters0, id, n_clusters = func_clust_target(dataset, name)
# clusters0, id, n_clusters = func_clust_drivers(dataset,dataset2,'inputs')

inputs,targets,indx = pre_training(dataset,dataset2,n_clusters,clusters0,name)

plotting_clusters(years,n_clusters,targets)

regr_all = []
scaler_inputs_all = []
scaler_targets_all = []

predictions = np.full(targets.shape,np.nan)
season_train, season_test = seasonality(n_clusters,targets,1)  # removing the seasonality for each cluster before training

for i in range (0,n_clusters):

    inputs2 = inputs[:,:,:,i] # inputs of the i cluster 
    targets2 = targets[:,:,i] - season_train[:,:,i] # targets of the i cluster    
    regr, scaler_inputs,scaler_targets = regressor(inputs2,targets[:,:,i],name,i)

    scaler_inputs_all.append(scaler_inputs)
    scaler_targets_all.append(scaler_targets)
    regr_all.append(regr)

    predictions[:,:,i] = scaling(regr_all[i],inputs2,scaler_inputs_all[i],targets2,scaler_targets_all[i]) # putting them in the right place

r_train,rms_train,slope_train = post_training(n_clusters,targets,predictions)

plotting_mean_values(years,n_clusters,targets,predictions,r_train,rms_train,slope_train,category,units,'Salish Sea (Training)')


## Other Years

In [ ]:
dataset = ds.sel(time_counter = slice('2021', '2024'))
dataset2 = ds2.sel(time_counter = slice('2021', '2024'))

dates = pd.DatetimeIndex(dataset['time_counter'].values)
years = np.unique(dataset.time_counter.dt.year)

indx = ~((dataset.time_counter.dt.month==2) & (dataset.time_counter.dt.day==29))
dates = dates[indx]

inputs,targets,indx = pre_training(dataset,dataset2,n_clusters,clusters0,name)

predictions = np.full(targets.shape,np.nan)

for i in range (0,n_clusters):

    inputs2 = inputs[:,:,:,i] # inputs of the i cluster
    targets2 = targets[:,:,i] - season_test[:,:,i] # targets of the i cluster

    predictions[:,:,i] = scaling(regr_all[i],inputs2,scaler_inputs_all[i],targets2,scaler_targets_all[i]) # putting them in the right place

    r_test, rms_test, slope_test = evaluation(dates,name,years,targets[:,:,i],predictions[:,:,i],i)

plotting_mean_values(years,n_clusters,targets,predictions,_,_,_,category,units,'Salish Sea')


## Saving

In [68]:
# if id == 0:
#     path = '/data/ibougoudis/MOAD/files/results/' + name + '/func_reg_cl_d2_target_b1/'
# else:
#     path = '/data/ibougoudis/MOAD/files/results/' + name + 'func_reg_cl_d2_drivers_b1/'

# os.makedirs(path, exist_ok=True)
# with lzma.open(path + 'regr_all.xz', 'wb') as f:
    
#     dill.dump(regr_all, f)

# with open(path + 'metrics.pkl', 'wb') as f:
#     dill.dump([r_train,rms_train,slope_train,dates_season,season], f)

# file_creation(path, targets_all, 'Targets')
# file_creation(path, predictions_all, 'Predictions')
# file_creation(path, (targets_all-predictions_all), 'Targets - Predictions')
